Huggingface part, put this in app.py in filed pulled from huggingface by

 When prompted for a password, use an access token with write permissions.
Generate one from your settings: https://huggingface.co/settings/tokens
git clone https://huggingface.co/spaces/felixfritzen/project
Modify the files locally, then commit and push:


 git commit -am "Update space"
 
 git push

In [78]:
import pickle
import hopsworks
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr

project = hopsworks.login(api_key_value="wOWQmzzHeITT5wlJ.FkOButuQ3XpDXSUO1LnTuaNfD7SPWolfF1hateoistsLFFsBF7upULn5z6qKPOIB")
fs = project.get_feature_store() 
mr = project.get_model_registry()

model = mr.get_model("KMeans_Model", version=1)
model_dir = model.download()
print(f"Model downloaded to: {model_dir}")
with open(f"{model_dir}/kmeans_model.pkl", "rb") as f:
    kmeans_model = pickle.load(f)

text_model = SentenceTransformer('all-MiniLM-L6-v2')

#encoded_headlines = model.encode(headline_df['title'].to_list())
#loaded from hopsworks!
fg_vectors = fs.get_feature_group(
    name='high_dimensional_vectors', 
    version=2,
)
retrieved_data = fg_vectors.read()
vectors = retrieved_data[[f"dim_{i}" for i in range(384)]].to_numpy()
headlines = retrieved_data['headlines']
print(vectors.shape, headlines.shape)

def df_between_dates(df, start_date, end_date):
    return df[(df['pubdate'] >= start_date) & (df['pubdate'] <= end_date)]

def get_most_similar_headlines(query_embedding, embeddings, headlines, top_n=5):
    print('compute similarities')
    similarities = cosine_similarity(query_embedding, embeddings)[0]
    top_indices = np.argsort(-similarities)[:top_n]
    return [(headlines[i], similarities[i]) for i in top_indices]

def compare(encoded_headlines, headlines, query, kmeans_model, text_model):
    encoding = text_model.encode([query])
    most_similar = get_most_similar_headlines(encoding, encoded_headlines, headlines, top_n=5)
    cluster = kmeans_model.predict(encoding)
    print(f"Query: {query}")
    print()
    outp = ""
    outp += str(f"Cluster: {cluster[0]}")
    outp += "\nMost Similar Headlines:"
    
    for i, (headline, similarity) in enumerate(most_similar, 1):
        outp +=f"\n{i}. {headline} (Similarity: {similarity:.4f})"
    return str(outp)
        
def apply_kmean(prompt):
    return compare(vectors, headlines, prompt, kmeans_model, text_model)

with gr.Blocks(css=".gradio-container {background-color: white;}") as demo:
    gr.Markdown("# Headline prediction app, [clustering page](https://felixfritzen.github.io/Scalable_proj/)")
    text_input = gr.Textbox(label="Enter your headline:")
    text_output = gr.Textbox(label="Most simmilar headlines:")
    gr.Button("Predict headlines").click(apply_kmean, inputs=text_input, outputs=text_output)
    gr.HTML('<iframe src="https://felixfritzen.github.io/Scalable_proj/" width="100%" height="100%" style="border:none;"></iframe>', min_height=50)

if __name__ == "__main__":
    demo.launch(share=True)

2024-12-13 00:33:53,889 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-13 00:33:53,900 INFO: Initializing external client
2024-12-13 00:33:53,906 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-13 00:33:54,922 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1163414
Model downloaded to: /var/tmp/68ebf8d6-c40d-4202-88b0-86e48b275d60/KMeans_Model/1
2024-12-13 00:33:57,068 INFO: Use pytorch device_name: cpu
2024-12-13 00:33:57,069 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.70s) 
(14220, 384) (14220,)
* Running on local URL:  http://127.0.0.1:7884
2024-12-13 00:34:05,472 INFO: HTTP Request: GET http://127.0.0.1:7884/gradio_api/startup-events "HTTP/1.1 200 OK"
2024-12-13 00:34:05,486 INFO: HTTP Request: HEAD http://127.0.0.1:7884/ "HTTP/1.1 200 OK"
2024-12-13 00:34:05,523 INFO: HTTP Request: GE

In [82]:
with gr.Blocks(css=".gradio-container {background-color: white;}") as demo:
    gr.Markdown("# Headline prediction app, [clustering page](https://felixfritzen.github.io/Scalable_proj/)")
    text_input = gr.Textbox(label="Enter your headline:")
    text_output = gr.Textbox(label="Most simmilar headlines:")
    gr.Button("Predict headlines").click(apply_kmean, inputs=text_input, outputs=text_output)
    gr.HTML('<iframe src="https://felixfritzen.github.io/Scalable_proj/" width="100%" height="100%" style="border:none;"></iframe>', min_height='1000')

if __name__ == "__main__":
    demo.launch(share=True)

2024-12-13 00:37:02,471 INFO: HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
* Running on local URL:  http://127.0.0.1:7888
2024-12-13 00:37:02,647 INFO: HTTP Request: GET http://127.0.0.1:7888/gradio_api/startup-events "HTTP/1.1 200 OK"
2024-12-13 00:37:02,661 INFO: HTTP Request: HEAD http://127.0.0.1:7888/ "HTTP/1.1 200 OK"
2024-12-13 00:37:03,263 INFO: HTTP Request: GET https://api.gradio.app/v3/tunnel-request "HTTP/1.1 200 OK"
* Running on public URL: https://9444491129ecc9c6e8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
2024-12-13 00:37:04,596 INFO: HTTP Request: HEAD https://9444491129ecc9c6e8.gradio.live "HTTP/1.1 200 OK"
